In [2]:
import requests
import json
import pandas as pd
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings

disable_warnings(InsecureRequestWarning)

base_url = "https://viperdb.scripps.edu"

all_tnums_url = "/services/tnumber_index.php?serviceName=tnumbers"
all_genuses_url = "/services/genus_index.php?serviceName=genus"
all_families_url = "/services/family_index.php?serviceName=families"

tnum_members_url = "/services/tnumber_index.php?serviceName=tnumber_members&tnumber="
genus_members_url = "/services/genus_index.php?serviceName=genus_members&genus="
family_members_url = "/services/family_index.php?serviceName=family_members&family="

def tnum_url(n):
    return base_url + tnum_members_url + str(n)
def genus_url(n):
    return base_url + genus_members_url + str(n)
def family_url(n):
    return base_url + family_members_url + str(n)

#sort list of dictionaries by resolution and take the n lowest
def lowres_dictlist(dictlist, n):
    lowres = {}
    for k in dictlist.keys():
        sortres = sorted(dictlist[k], key=lambda d: (d['resolution'] is None, d['resolution']))
        lowres[k] = sortres[:n]
    return lowres

#get lists of all tnums, genuses, families in VDB
tnums_res = requests.get((base_url + all_tnums_url), verify=False)
tnums = json.loads(tnums_res.text)

tnum_members = {}
for i in range(len(tnums)):
    tnum = tnums[i]["tnumber"]
    members_res = requests.get(tnum_url(tnum), verify=False)
    tnum_members[tnum] = json.loads(members_res.text)

ConnectionError: HTTPSConnectionPool(host='viperdb.scripps.edu', port=443): Max retries exceeded with url: /services/tnumber_index.php?serviceName=tnumbers (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11edac070>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
genus_dict = {g:[v for v in parv_members if v['genus'] == g] for g in parv_genuses}

genus_lowres = lowres_dictlist(genus_dict, 1)

pdb_ids = [genus_lowres[g][0]['entry_id'] for g in genus_lowres.keys()]

In [4]:
id_res = {v[0]['entry_id']:v[0]['resolution'] for v in genus_lowres.values()}

In [2]:
import requests
import json
import pandas as pd
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings

disable_warnings(InsecureRequestWarning)

base_url = "https://viperdb.scripps.edu"

all_tnums_url = "/services/tnumber_index.php?serviceName=tnumbers"
all_genuses_url = "/services/genus_index.php?serviceName=genus"
all_families_url = "/services/family_index.php?serviceName=families"

tnum_members_url = "/services/tnumber_index.php?serviceName=tnumber_members&tnumber="
genus_members_url = "/services/genus_index.php?serviceName=genus_members&genus="
family_members_url = "/services/family_index.php?serviceName=family_members&family="

def tnum_url(n):
    return base_url + tnum_members_url + str(n)
def genus_url(n):
    return base_url + genus_members_url + str(n)
def family_url(n):
    return base_url + family_members_url + str(n)

#sort list of dictionaries by resolution and take the n lowest
def lowres_dictlist(dictlist, n):
    lowres = {}
    for k in dictlist.keys():
        sortres = sorted(dictlist[k], key=lambda d: (d['resolution'] is None, d['resolution']))
        lowres[k] = sortres[:n]
    return lowres

#get lists of all tnums, genuses, families in VDB
families_res = requests.get((base_url + all_families_url), verify=False)
families = json.loads(families_res.text)

#get lowest resolution structure for each genus in parvoviridae
family_members = {}
for i in range(len(families)):
    fam = families[i]["family"]
    members_res = requests.get(family_url(fam), verify=False)
    family_members[fam] = json.loads(members_res.text)

parv_members = family_members['Parvoviridae']

parv_genuses = list(set([v['genus'] for v in parv_members]))    



In [6]:
genus_members = {g:[v['resolution'] for v in value] for g, value in tnum_.items()}

In [7]:
genus_members

{'Ambidensovirus': ['4mgu'],
 'Dependoparovirus': ['7ltm'],
 'Protoparvovirus': ['7m3m'],
 'Virus Like Particle': ['6nf9'],
 'Denpendoparvovirus': ['6jct'],
 'Dependoparvovirus': ['7kfr'],
 'Erythroparvovirus': ['6nn3'],
 'Penstyldensovirus': ['3n7x'],
 'Densovirus': ['6wh3'],
 'Bocaparvovirus': ['7l0x'],
 'Parvovirus': ['4g0r']}

In [45]:
pdb_ids

['4g0r',
 '3n7x',
 '7m3m',
 '6jct',
 '7ltm',
 '6nn3',
 '7l0x',
 '7kfr',
 '4mgu',
 '6wh3',
 '6nf9']

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

options = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "/Users/kageyamatobio/vpscrape/getvirions/capsids/"}
options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path='./chromedriver',options=options)

base_url = 'https://viperdb.scripps.edu/Info_Page.php?VDB='
for id in pdb_ids:
    try:
        res = driver.get(base_url + id)
        while True:
            try:
                dlds = driver.find_element(By.ID, "btn-downloads")
                dlds.click()
                dlbtn = driver.find_element(By.LINK_TEXT, "VIPER Coordinates")
                dlbtn.click()
                time.sleep(1)
                break
            except:
                break
    except Exception as e:
        print(e)
driver.close()

import subprocess
subprocess.check_call(['./pdb.sh', 'capsids'])

In [165]:
fsf_one = []
klist = list(fsf_lowres.keys())
ids = []
for i in range(len(list(fsf_lowres.values()))):
    try:
        added = False
        vlist = list(list(fsf_lowres.values())[i].values())[0]
        for v in vlist:
            if v['entry_id'] not in ids:
                fsf_one.append(v)
                ids.append(v['entry_id'])
                added = True
                break
        if not added:
            fsf_one.append(vlist[0])
            added = True
        if added: fsf_one[-1]["fold"] = klist[i]
    except Exception as e:
        print(e)

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [168]:
df = pd.DataFrame.from_dict(fsf_one)

In [170]:
df.to_excel('fsf_structs.xlsx')

In [61]:
df = pd.DataFrame.from_dict(tnum_structs)
df.to_excel('fsf_structs.xlsx')

TypeError: from_dict() got an unexpected keyword argument 'index'

In [58]:
#make dict with all members for each tnum/genus/family
tnum_members = {}
for i in range(len(tnums)):
    tnum = tnums[i]["tnumber"]
    members_res = requests.get(tnum_url(tnum), verify=False)
    tnum_members[tnum] = json.loads(members_res.text)

genus_members = {}
for i in range(len(genuses)):
    genus = genuses[i]["genus"]
    members_res = requests.get(genus_url(genus), verify=False)
    genus_members[genus] = json.loads(members_res.text)

family_members = {}
for i in range(len(families)):
    family = families[i]["family"]
    members_res = requests.get(family_url(family), verify=False)
    family_members[family] = json.loads(members_res.text)
    
#make the dictlists only contain the n lowest res structures
n_structures = 5
tnum_lowres = lowres_dictlist(tnum_members, n_structures)
genus_lowres = lowres_dictlist(genus_members, n_structures)
family_lowres = lowres_dictlist(family_members, n_structures)

#get families containing each FSF
file_url = 't2.txt'
fsf_families = {}
with open(file_url, 'r') as f:
    lines = f.readlines()
    for l in lines:
        l = l.split()
        fams = [x.replace(',', '') for x in l[1:]]
        fsf_families[l[0]] =fams 

#make dict of each FSF to n lowest res structures from each associated family
fsf_lowres = {}
for k, v in fsf_families.items():
    members = {}
    for f in v:
        try: 
            members[f] = family_lowres[f]
        except:
            pass
    fsf_lowres[k] = members

#count structs

tnum_structs = []
tnum_excluded = []
for k in tnum_members:
    for t in tnum_members[k]:
        if t['resolution'] is None or t['family'] == 'Unclassified' or t['genus'] == 'Unclassified':
            tnum_excluded.append(t)
        else:
            tnum_structs.append(t['entry_id'])
len(set(tnum_structs))

#count num for each tnum

for k in tnum_members:
    s = set([t['entry_id'] if t['resolution'] is not None else '' for t in tnum_members[k]])
    #print('tnum: ',k)
    #print(len(s))

tnum:  1
261
tnum:  13
24
tnum:  16
12
tnum:  169
1
tnum:  2
28
tnum:  277
0
tnum:  28d
3
tnum:  3
260
tnum:  4
88
tnum:  43
1
tnum:  7
2
tnum:  7d
26
tnum:  7l
43
tnum:  9
5
tnum:  NA
0
tnum:  pT169
1
tnum:  pT21
2
tnum:  pT25
15
tnum:  pT27
2
tnum:  pT3
443
tnum:  pT31
1


In [26]:
tnum_structs = []
tnum_excluded = []
for k in tnum_members:
    for t in tnum_members[k]:
        if t['resolution'] is None or t['family'] == 'Unclassified' or t['genus'] == 'Unclassified':
            tnum_excluded.append(t)
        else:
            t['tnum'] = k
            tnum_structs.append(t)

In [18]:
df = pd.DataFrame.from_dict(tnum_structs, index=None)

df.to_excel('tnums.xlsx')